In [3]:
import joblib
import os
import pandas as pd
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.robustness import test_model_robustness

pipeline = joblib.load("../models/best_tuned_model.joblib")
df = pd.read_csv("../data/clean_dataset.csv")

numeric_features = df.drop(columns=["CLASS_LABEL"]).columns.tolist()


In [4]:
test_email = """
Your account has been suspended!
Verify immediately:
http://malicious-paypal-security.com/login/secure
"""

results = test_model_robustness(pipeline, test_email, numeric_features)

results


{'original': {'error': "'NumLetters'"},
 'homoglyph': {'prediction': 0, 'probability': 0.275},
 'no_spaces': {'error': "'NumLetters'"},
 'html_wrapped': {'error': "'NumLetters'"},
 'random_insert': {'error': "'NumLetters'"}}

In [5]:
import json, os
os.makedirs("../results/robustness", exist_ok=True)

with open("../results/robustness/day9_report.json", "w") as f:
    json.dump(results, f, indent=2)

print("Saved robustness report.")


Saved robustness report.


In [6]:
print("Classes:", pipeline.classes_)

Classes: [0 1]


In [8]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/clean_dataset.csv")

X = df.drop(columns=["CLASS_LABEL"])
y = df["CLASS_LABEL"]

# IMPORTANT — we preserve the DataFrame form
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(X_train_df.shape, X_test_df.shape)


(7664, 48) (1917, 48)


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix

# X_test_df is the DataFrame used for testing (same columns/order)
y_pred = pipeline.predict(X_test_df)
y_proba = pipeline.predict_proba(X_test_df)
# get phishing prob
phish_index = list(pipeline.classes_).index(1)
phish_probs = y_proba[:, phish_index]

# Count inconsistent cases where pred != (phish_prob >= 0.5)
pred_by_thresh = (phish_probs >= 0.5).astype(int)
inconsistent = (y_pred != pred_by_thresh).sum()
print("Inconsistent predictions vs 0.5-threshold:", inconsistent)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion:\n", confusion_matrix(y_test, y_pred))


Inconsistent predictions vs 0.5-threshold: 0
Accuracy: 1.0
Confusion:
 [[1000    0]
 [   0  917]]
